In [1]:
from cdapython import Q, columns, unique_terms
import cdapython
import pandas as pd
from collections import Counter
print(cdapython.__file__)
print(cdapython.__version__)

/Users/elijahlowe/opt/miniconda3/envs/cda/lib/python3.9/site-packages/cdapython/__init__.py
2022.2.23


# Find files from TCGA-BRCA project, with donors over the age of 50 with imaging data

The first step is to explore the searchable field known as "columns" in our system by running the` columns()` command:

In [12]:
columns()

['id',
 'identifier',
 'identifier.system',
 'identifier.value',
 'species',
 'sex',
 'race',
 'ethnicity',
 'days_to_birth',
 'subject_associated_project',
 'vital_status',
 'age_at_death',
 'cause_of_death',
 'File',
 'File.id',
 'File.identifier',
 'File.identifier.system',
 'File.identifier.value',
 'File.label',
 'File.data_category',
 'File.data_type',
 'File.file_format',
 'File.associated_project',
 'File.drs_uri',
 'File.byte_size',
 'File.checksum',
 'File.data_modality',
 'File.imaging_modality',
 'File.dbgap_accession_number',
 'ResearchSubject',
 'ResearchSubject.id',
 'ResearchSubject.identifier',
 'ResearchSubject.identifier.system',
 'ResearchSubject.identifier.value',
 'ResearchSubject.member_of_research_project',
 'ResearchSubject.primary_diagnosis_condition',
 'ResearchSubject.primary_diagnosis_site',
 'ResearchSubject.Diagnosis',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.identifier',
 'ResearchSubject.Diagnosis.identifier.system',
 'ResearchSubjec

From looking at the terms found in columns we can see that 'File.associated_project' and 'days_to_birth'

In [13]:
unique_terms('File.data_category')

['Biospecimen',
 'Clinical',
 'Combined Nucleotide Variation',
 'Copy Number Variation',
 'DNA Methylation',
 'Imaging',
 'Peptide Spectral Matches',
 'Processed Mass Spectra',
 'Protein Assembly',
 'Proteome Profiling',
 'Quality Metrics',
 'Raw Mass Spectra',
 'Sequencing Reads',
 'Simple Nucleotide Variation',
 'Somatic Structural Variation',
 'Structural Variation',
 'Transcriptome Profiling']

In [ ]:
unique_terms()

In [38]:
q1 = Q('File.associated_project = "tcga_brca"')
q2 = Q('File.associated_project = "TCGA-BRCA"')
q3 = Q('days_to_birth < -50*365')
q4 = Q('File.data_category = "Imaging"')
q = q4.And(q3.And(q1.Or(q2)))

In [41]:
q1 = Q('File.associated_project = "TCGA-BRCA"')
q2 = Q('days_to_birth < -50*365')
q3 = Q('identifier.system = "IDC"') #because of the naming difference you have to use a different column to search
q = q3.And(q1.And(q2))

In [47]:
q = q3.And(q1.And(q2))
count = q.counts(host="http://35.192.60.10:8080/")
count

(500)
Reason: 
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Tue, 01 Mar 2022 20:18:33 GMT', 'Connection': 'close'})
HTTP response body: {"message":"Unrecognized name: _identifier; Did you mean identifier? at [37:9]","statusCode":500,"causes":["Unrecognized name: _identifier; Did you mean identifier? at [37:9]","400 Bad Request\nGET https://www.googleapis.com/bigquery/v2/projects/gdc-bq-sample/queries/fff86d2f-5b93-4acb-a9d5-c5f8546f447f?location=US&maxResults=0&prettyPrint=false\n{\n  \"code\" : 400,\n  \"errors\" : [ {\n    \"domain\" : \"global\",\n    \"location\" : \"q\",\n    \"locationType\" : \"parameter\",\n    \"message\" : \"Unrecognized name: _identifier; Did you mean identifier? at [37:9]\",\n    \"reason\" : \"invalidQuery\"\n  } ],\n  \"message\" : \"Unrecognized name: _identifier; Did you mean identifier? at [37:9]\",\n  \"status\" : \"INVALID_ARGUMENT\"\n}"]}



In [45]:
r = q.run()

Getting results from database

Total execution time: 34783 ms


In [46]:
r


        QueryID: cf65600d-c9ea-4f42-973b-df4189c0480e
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(identifier) AS _identifier, UNNEST(File) AS _File WHERE ((_identifier.system = 'IDC') AND ((_File.associated_project = 'TCGA-BRCA') AND (all_v2_1.days_to_birth < -50*365)))
        Offset: 0
        Count: 100
        Total Row Count: 5167
        GDC Count: 5948 
 	PDC Count: 1356 
 	IDC Count: 116488
        More pages: True
        

Below I'm using pandas json_normalize function to flatten the data and observe the files. You get an idea of the overall data structure by looking at the flatten format, and any columns that has a list/nested structure can be further explored. If you look at the output below you will see that there are `GDC` files.

In [48]:
pd.json_normalize(r).head(6)

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death,File,ResearchSubject
0,TCGA-BH-A18F,"[{'system': 'GDC', 'value': 'TCGA-BH-A18F'}, {...",Homo sapiens,female,white,not hispanic or latino,-18594,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '14f27a13-f309-4215-a33f-e78b06ba3f30'...,[{'id': 'bfd49783-1767-469b-9d79-7822301c5efc'...
1,TCGA-BH-A18F,"[{'system': 'GDC', 'value': 'TCGA-BH-A18F'}, {...",Homo sapiens,female,white,not hispanic or latino,-18594,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '14f27a13-f309-4215-a33f-e78b06ba3f30'...,[{'id': 'bfd49783-1767-469b-9d79-7822301c5efc'...
2,TCGA-BH-A0B1,"[{'system': 'GDC', 'value': 'TCGA-BH-A0B1'}, {...",Homo sapiens,female,white,not reported,-24315,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '62d4515f-a30b-4b1a-b2dd-c8bf9476e803'...,[{'id': 'b4e4fea2-b802-4a86-923e-3fcbd3c708fe'...
3,TCGA-E2-A1IJ,"[{'system': 'GDC', 'value': 'TCGA-E2-A1IJ'}, {...",Homo sapiens,female,white,not hispanic or latino,-20956,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '56fc7bb8-ac12-45d0-8c28-69652728a74c'...,[{'id': 'b09643c2-ded5-4346-b3c5-90a48f88a02e'...
4,TCGA-AO-A0JI,"[{'system': 'GDC', 'value': 'TCGA-AO-A0JI'}, {...",Homo sapiens,female,white,not hispanic or latino,-20694,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '32d96a80-9060-42ed-beb3-c6662c5c2265'...,[{'id': 'e17c565c-9857-4df3-a352-903b53093c85'...
5,TCGA-AO-A0JI,"[{'system': 'GDC', 'value': 'TCGA-AO-A0JI'}, {...",Homo sapiens,female,white,not hispanic or latino,-20694,"[TCGA-BRCA, tcga_brca]",Alive,None,None,[{'id': '32d96a80-9060-42ed-beb3-c6662c5c2265'...,[{'id': 'e17c565c-9857-4df3-a352-903b53093c85'...


In [49]:
pd.json_normalize(r, record_path =['ResearchSubject']).head(6)

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,Diagnosis,File,Specimen
0,bfd49783-1767-469b-9d79-7822301c5efc,"[{'system': 'GDC', 'value': 'bfd49783-1767-469...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': '43d2ccb2-20e6-5035-aa2a-bbc174ec3d1e'...,[{'id': '14f27a13-f309-4215-a33f-e78b06ba3f30'...,[{'id': 'adbf4056-e7d4-4282-8771-f2d6029386ab'...
1,bfd49783-1767-469b-9d79-7822301c5efc,"[{'system': 'GDC', 'value': 'bfd49783-1767-469...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': '43d2ccb2-20e6-5035-aa2a-bbc174ec3d1e'...,[{'id': '14f27a13-f309-4215-a33f-e78b06ba3f30'...,[{'id': 'adbf4056-e7d4-4282-8771-f2d6029386ab'...
2,b4e4fea2-b802-4a86-923e-3fcbd3c708fe,"[{'system': 'GDC', 'value': 'b4e4fea2-b802-4a8...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': 'de864803-01f0-56fa-96d8-2a832e7b5354'...,[{'id': '62d4515f-a30b-4b1a-b2dd-c8bf9476e803'...,[{'id': '31532d08-cb6e-40fa-9dfe-d2c6d037dcb1'...
3,b09643c2-ded5-4346-b3c5-90a48f88a02e,"[{'system': 'GDC', 'value': 'b09643c2-ded5-434...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': '1a64453a-09de-500f-a97a-945bf7cb6a56'...,[{'id': '56fc7bb8-ac12-45d0-8c28-69652728a74c'...,[{'id': '0c61c676-2a72-4dbe-8dcb-284bd112570c'...
4,e17c565c-9857-4df3-a352-903b53093c85,"[{'system': 'GDC', 'value': 'e17c565c-9857-4df...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': '55f24ee5-2989-5fbc-8c50-80f9b5d82e2b'...,[{'id': '32d96a80-9060-42ed-beb3-c6662c5c2265'...,[{'id': 'c48d1725-4af6-4fab-8066-8bf70a73b5b9'...
5,e17c565c-9857-4df3-a352-903b53093c85,"[{'system': 'GDC', 'value': 'e17c565c-9857-4df...",TCGA-BRCA,Ductal and Lobular Neoplasms,Breast,[{'id': '55f24ee5-2989-5fbc-8c50-80f9b5d82e2b'...,[{'id': '32d96a80-9060-42ed-beb3-c6662c5c2265'...,[{'id': 'c48d1725-4af6-4fab-8066-8bf70a73b5b9'...


In [62]:
pd.json_normalize(r, record_path =['ResearchSubject','Specimen','File'])

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number
0,053f01ed-3154-4aea-9e7f-932c435034b3,"[{'system': 'GDC', 'value': '053f01ed-3154-4ae...",TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None
1,092aea33-c867-4580-8a80-628da99124cd,"[{'system': 'GDC', 'value': '092aea33-c867-458...",5a4ccc96-fd1b-4237-b8f1-1344b880fa00.FPKM.txt.gz,Transcriptome Profiling,Gene Expression Quantification,TXT,TCGA-BRCA,drs://dg.4DFC:092aea33-c867-4580-8a80-628da991...,506320,12fef90dad96197718f9aa459f8a3338,Genomic,None,None
2,144c0c29-83ec-48bb-877f-cec531ba0943,"[{'system': 'GDC', 'value': '144c0c29-83ec-48b...",TCGA-BH-A18F-01A-21-A13F-20_RPPA_data.tsv,Proteome Profiling,Protein Expression Quantification,TSV,TCGA-BRCA,drs://dg.4DFC:144c0c29-83ec-48bb-877f-cec531ba...,22350,5d9cd7a5a43f876ea4825fa3623d656e,Genomic,None,None
3,14f27a13-f309-4215-a33f-e78b06ba3f30,"[{'system': 'GDC', 'value': '14f27a13-f309-421...",TCGA-BH-A18F-01A-11R-A12C-13_mirna_gdc_realn.bam,Sequencing Reads,Aligned Reads,BAM,TCGA-BRCA,drs://dg.4DFC:14f27a13-f309-4215-a33f-e78b06ba...,48158585,97a85067371cb060f7f9796c86945845,Genomic,None,None
4,158a3fce-f32b-4e5d-95ae-84977aa4fea9,"[{'system': 'GDC', 'value': '158a3fce-f32b-4e5...",TCGA-BRCA.df0ea333-0f5a-4da4-9242-f74a28e6a9ac...,Copy Number Variation,Allele-specific Copy Number Segment,TXT,TCGA-BRCA,drs://dg.4DFC:158a3fce-f32b-4e5d-95ae-84977aa4...,5703,78cca4a25a887d7e84dd1b7d0192d93a,Genomic,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29436,daac3326-5cbe-11e9-849f-005056921935,"[{'system': 'PDC', 'value': 'daac3326-5cbe-11e...",TCGA_BH-A0EE_AO-A0J9_BH-A0E0_117C_W_BI_2013041...,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC-TCGA,drs://dg.4DFC:daac3326-5cbe-11e9-849f-00505692...,1162360468,599180659ae56f0a80ca1d670687fd3d,Proteomic,None,None
29437,e2f1737a-5cbe-11e9-849f-005056921935,"[{'system': 'PDC', 'value': 'e2f1737a-5cbe-11e...",TCGA_BH-A0EE_AO-A0J9_BH-A0E0_117C_W_BI_2013041...,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC-TCGA,drs://dg.4DFC:e2f1737a-5cbe-11e9-849f-00505692...,1128557103,2e52d14c428b97edc855178854732dba,Proteomic,None,None
29438,e81586a2-5cbe-11e9-849f-005056921935,"[{'system': 'PDC', 'value': 'e81586a2-5cbe-11e...",TCGA_BH-A0EE_AO-A0J9_BH-A0E0_117C_W_BI_2013041...,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC-TCGA,drs://dg.4DFC:e81586a2-5cbe-11e9-849f-00505692...,774177623,7d8615e0bea063da862daf5cb369c900,Proteomic,None,None
29439,f429400a-5d36-11e9-8f80-005056921935,"[{'system': 'PDC', 'value': 'f429400a-5d36-11e...",TCGA_BH-A0EE_AO-A0J9_BH-A0E0_117C_P_BI_2013042...,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC-TCGA,drs://dg.4DFC:f429400a-5d36-11e9-8f80-00505692...,1513605463,b889cddcdfe1170f585cbcbb0a826c8d,Proteomic,None,None


In [59]:
df[df['specimen_type'] == 'sample']

,id,identifier,associated_project,age_at_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,derived_from_subject,File
0,adbf4056-e7d4-4282-8771-f2d6029386ab,"[{'system': 'GDC', 'value': 'adbf4056-e7d4-428...",TCGA-BRCA,-18594,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-BH-A18F,[{'id': '053f01ed-3154-4aea-9e7f-932c435034b3'...
15,065a531a-14bd-4714-8283-cd9ccf69b66f,"[{'system': 'GDC', 'value': '065a531a-14bd-471...",TCGA-BRCA,-18594,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-BH-A18F,[{'id': 'a5e25681-8800-4d3d-a202-35d2216175c6'...
18,3fb05189-c4d5-437f-93e6-9e333fa05e44,"[{'system': 'GDC', 'value': '3fb05189-c4d5-437...",TCGA-BRCA,-18594,Ductal and Lobular Neoplasms,None,Solid Tissue Normal,sample,initial specimen,TCGA-BH-A18F,[{'id': '053f01ed-3154-4aea-9e7f-932c435034b3'...
29,adbf4056-e7d4-4282-8771-f2d6029386ab,"[{'system': 'GDC', 'value': 'adbf4056-e7d4-428...",TCGA-BRCA,-18594,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-BH-A18F,[{'id': '053f01ed-3154-4aea-9e7f-932c435034b3'...
44,065a531a-14bd-4714-8283-cd9ccf69b66f,"[{'system': 'GDC', 'value': '065a531a-14bd-471...",TCGA-BRCA,-18594,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-BH-A18F,[{'id': 'a5e25681-8800-4d3d-a202-35d2216175c6'...
...,...,...,...,...,...,...,...,...,...,...,...
3006,2c740d6c-6420-11e8-bcf1-0a2705229b82,"[{'system': 'PDC', 'value': '2c740d6c-6420-11e...",CPTAC-TCGA,-22642,Breast Invasive Carcinoma,Not Reported,Primary Tumor,sample,initial specimen,TCGA-AO-A0J9,[{'id': '0930fbfa-5d37-11e9-8f80-005056921935'...
3008,6527a955-d832-4d06-98a9-24cefb32e227,"[{'system': 'GDC', 'value': '6527a955-d832-4d0...",TCGA-BRCA,-22642,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-AO-A0J9,[{'id': '91f55a25-ae69-4c98-8b31-326b50ecdb26'...
3011,0c8df249-80de-4675-93e5-d06ad8ea3001,"[{'system': 'GDC', 'value': '0c8df249-80de-467...",TCGA-BRCA,-22642,Ductal and Lobular Neoplasms,None,Primary Tumor,sample,initial specimen,TCGA-AO-A0J9,[{'id': '053f01ed-3154-4aea-9e7f-932c435034b3'...
3033,c1d0e9fb-eab0-40cd-9dd8-190bd6b5e180,"[{'system': 'GDC', 'value': 'c1d0e9fb-eab0-40c...",TCGA-BRCA,-22642,Ductal and Lobular Neoplasms,None,Blood Derived Normal,sample,initial specimen,TCGA-AO-A0J9,[{'id': '053f01ed-3154-4aea-9e7f-932c435034b3'...


In [63]:
pd.json_normalize(r, record_path =['ResearchSubject','File'],
                  meta=['id','species','sex','race','ethnicity',['ResearchSubject','id']],
                  meta_prefix= 'subject_', max_level = 0,errors='ignore').drop('identifier',1).sort_values('subject_id')

,id,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,subject_id,subject_species,subject_sex,subject_race,subject_ethnicity,subject_ResearchSubject.id
6663,d0c967b5-78ee-4bc4-9493-3cbe44af1bf5,nationwidechildrens.org_biospecimen_analyte_br...,Biospecimen,Biospecimen Supplement,BCR Biotab,TCGA-BRCA,drs://dg.4DFC:d0c967b5-78ee-4bc4-9493-3cbe44af...,1728468,103577529e4355cfd0a5da055ea5a403,Genomic,None,None,TCGA-AO-A0J8,Homo sapiens,female,white,not hispanic or latino,02bf5203-f9cd-4c5a-97b4-e5584dc22325
6716,433ebcbd-716b-4c5f-b281-cf5774e40dbe,456bf061-467d-4bd8-bf71-b0a54ef06dea.FPKM.txt.gz,Transcriptome Profiling,Gene Expression Quantification,TXT,TCGA-BRCA,drs://dg.4DFC:433ebcbd-716b-4c5f-b281-cf5774e4...,523216,423e901c64aeb96ee138808bb3e147ce,Genomic,None,None,TCGA-AO-A0J8,Homo sapiens,female,white,not hispanic or latino,02bf5203-f9cd-4c5a-97b4-e5584dc22325
6715,1e6b79ff-9787-4cbe-b19d-ebabb6b43589,nationwidechildrens.org_clinical_drug_brca.txt,Clinical,Clinical Supplement,BCR Biotab,TCGA-BRCA,drs://dg.4DFC:1e6b79ff-9787-4cbe-b19d-ebabb6b4...,961048,a83b5130bd671998aa1ec339a542121d,Genomic,None,None,TCGA-AO-A0J8,Homo sapiens,female,white,not hispanic or latino,02bf5203-f9cd-4c5a-97b4-e5584dc22325
6714,83c3eaed-dcfa-4d36-af39-debcf8409927,456bf061-467d-4bd8-bf71-b0a54ef06dea.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TXT,TCGA-BRCA,drs://dg.4DFC:83c3eaed-dcfa-4d36-af39-debcf840...,256657,5208eb57681337fdeb7b07fa285c527d,Genomic,None,None,TCGA-AO-A0J8,Homo sapiens,female,white,not hispanic or latino,02bf5203-f9cd-4c5a-97b4-e5584dc22325
6713,f9ff49e9-31ce-4051-9157-4bf4b0c5859a,TCGA.BRCA.somaticsniper.f9ff49e9-31ce-4051-915...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:f9ff49e9-31ce-4051-9157-4bf4b0c5...,621054920,a5d844676518cfe8517d86cccf588ccc,Genomic,None,None,TCGA-AO-A0J8,Homo sapiens,female,white,not hispanic or latino,02bf5203-f9cd-4c5a-97b4-e5584dc22325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5253,8162d394-8b64-4da2-9f5b-d164c54b9608,nationwidechildrens.org_clinical_patient_brca.txt,Clinical,Clinical Supplement,BCR Biotab,TCGA-BRCA,drs://dg.4DFC:8162d394-8b64-4da2-9f5b-d164c54b...,1700943,f271f56d44b62d41bd6ddb0d26e525b4,Genomic,None,None,TCGA-OL-A66K,Homo sapiens,female,white,not hispanic or latino,2ab1b35b-9e56-475c-ac9b-15ed0a422683
5254,945ed04d-517b-46ec-8be3-56acb6201086,BURSA_p_TCGA_303_304_305_N_GenomeWideSNP_6_D12...,Copy Number Variation,Masked Copy Number Segment,TXT,TCGA-BRCA,drs://dg.4DFC:945ed04d-517b-46ec-8be3-56acb620...,9012,b28c02406e38e8c8f3d25276bb8c16f2,Genomic,None,None,TCGA-OL-A66K,Homo sapiens,female,white,not hispanic or latino,2ab1b35b-9e56-475c-ac9b-15ed0a422683
5255,f434b393-9e32-4ca2-aa1a-3b0de36b5ffe,nationwidechildrens.org_clinical_omf_v4.0_brca...,Clinical,Clinical Supplement,BCR Biotab,TCGA-BRCA,drs://dg.4DFC:f434b393-9e32-4ca2-aa1a-3b0de36b...,39381,4f6606f6b12f952e0339a40396cd7c92,Genomic,None,None,TCGA-OL-A66K,Homo sapiens,female,white,not hispanic or latino,2ab1b35b-9e56-475c-ac9b-15ed0a422683
5242,07e24277-17a0-4442-9d56-b235c93e4142,TCGA-OL-A66K-01Z-00-DX1.C1DC85F1-4FAE-4411-988...,Biospecimen,Slide Image,SVS,TCGA-BRCA,drs://dg.4DFC:07e24277-17a0-4442-9d56-b235c93e...,3401266790,246ecfccfdfda3254c50f14f3082a4ac,Genomic,None,None,TCGA-OL-A66K,Homo sapiens,female,white,not hispanic or latino,2ab1b35b-9e56-475c-ac9b-15ed0a422683


In [199]:
df[df.duplicated(keep=False)].sort_values('id')

,id,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,subject_species,subject_sex,subject_race,subject_ethnicity
8436,053f01ed-3154-4aea-9e7f-932c435034b3,TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None,Homo sapiens,female,white,not hispanic or latino
23735,053f01ed-3154-4aea-9e7f-932c435034b3,TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None,Homo sapiens,female,white,not reported
28053,053f01ed-3154-4aea-9e7f-932c435034b3,TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None,Homo sapiens,female,white,not hispanic or latino
20006,053f01ed-3154-4aea-9e7f-932c435034b3,TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None,Homo sapiens,female,white,not hispanic or latino
854,053f01ed-3154-4aea-9e7f-932c435034b3,TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c4...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c4350...,1882061658,ae12bbce7abcc03eff228935fa8d3d22,Genomic,None,None,Homo sapiens,female,black or african american,not hispanic or latino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23785,f9ff49e9-31ce-4051-9157-4bf4b0c5859a,TCGA.BRCA.somaticsniper.f9ff49e9-31ce-4051-915...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:f9ff49e9-31ce-4051-9157-4bf4b0c5...,621054920,a5d844676518cfe8517d86cccf588ccc,Genomic,None,None,Homo sapiens,female,white,not reported
26591,f9ff49e9-31ce-4051-9157-4bf4b0c5859a,TCGA.BRCA.somaticsniper.f9ff49e9-31ce-4051-915...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:f9ff49e9-31ce-4051-9157-4bf4b0c5...,621054920,a5d844676518cfe8517d86cccf588ccc,Genomic,None,None,Homo sapiens,female,white,not reported
25443,f9ff49e9-31ce-4051-9157-4bf4b0c5859a,TCGA.BRCA.somaticsniper.f9ff49e9-31ce-4051-915...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:f9ff49e9-31ce-4051-9157-4bf4b0c5...,621054920,a5d844676518cfe8517d86cccf588ccc,Genomic,None,None,Homo sapiens,female,white,not hispanic or latino
7280,f9ff49e9-31ce-4051-9157-4bf4b0c5859a,TCGA.BRCA.somaticsniper.f9ff49e9-31ce-4051-915...,Simple Nucleotide Variation,Aggregated Somatic Mutation,MAF,TCGA-BRCA,drs://dg.4DFC:f9ff49e9-31ce-4051-9157-4bf4b0c5...,621054920,a5d844676518cfe8517d86cccf588ccc,Genomic,None,None,Homo sapiens,female,white,not hispanic or latino


An even closer look revels thasort_values also `PDC` files.

In [150]:
pd.json_normalize(r, record_path =['File','identifier'],meta=['species','sex','race','ethnicity'],meta_prefix= 'subject_', max_level = 1,errors='ignore')['system'].unique()

array(['GDC', 'IDC', 'PDC'], dtype=object)

In [157]:
pd.json_normalize(r, record_path =['File'],meta=['species','sex','race','ethnicity'],meta_prefix= 'subject_', max_level = 1,errors='ignore').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30322 entries, 0 to 30321
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      30322 non-null  object
 1   identifier              30322 non-null  object
 2   label                   30322 non-null  object
 3   data_category           30322 non-null  object
 4   data_type               1489 non-null   object
 5   file_format             30322 non-null  object
 6   associated_project      30322 non-null  object
 7   drs_uri                 30322 non-null  object
 8   byte_size               1489 non-null   object
 9   checksum                1489 non-null   object
 10  data_modality           30322 non-null  object
 11  imaging_modality        28833 non-null  object
 12  dbgap_accession_number  0 non-null      object
 13  subject_species         30322 non-null  object
 14  subject_sex             30322 non-null  object
 15  su

In [18]:
# Filter python objects with list comprehensions
r2 = [file for subject in r for file in subject['File'] for ident in file['identifier'] if file['associated_project'] == 'tcga_brca' and ident['system'] == 'IDC']
# Transform python object back into json
#output_json = json.dumps(output_dict)
#[r3] = 
r2
# Show json
#print(output_json)

[{'id': '88209015-232a-478b-a4c1-0688fe4cf973',
  'identifier': [{'system': 'IDC',
    'value': '88209015-232a-478b-a4c1-0688fe4cf973'}],
  'label': '88209015-232a-478b-a4c1-0688fe4cf973.dcm',
  'data_category': 'Imaging',
  'data_type': None,
  'file_format': 'DICOM',
  'associated_project': 'tcga_brca',
  'drs_uri': 'gs://idc-open/88209015-232a-478b-a4c1-0688fe4cf973.dcm',
  'byte_size': None,
  'checksum': None,
  'data_modality': 'Imaging',
  'imaging_modality': 'MR',
  'dbgap_accession_number': None},
 {'id': '1005692c-ea63-4635-9ff6-bbb5359417c6',
  'identifier': [{'system': 'IDC',
    'value': '1005692c-ea63-4635-9ff6-bbb5359417c6'}],
  'label': '1005692c-ea63-4635-9ff6-bbb5359417c6.dcm',
  'data_category': 'Imaging',
  'data_type': None,
  'file_format': 'DICOM',
  'associated_project': 'tcga_brca',
  'drs_uri': 'gs://idc-open/1005692c-ea63-4635-9ff6-bbb5359417c6.dcm',
  'byte_size': None,
  'checksum': None,
  'data_modality': 'Imaging',
  'imaging_modality': 'MR',
  'dbgap_ac

In [164]:
pd.json_normalize(r2, record_path =['identifier'], meta= ['associated_project'], max_level = 1,errors='ignore')

,system,value,associated_project
0,IDC,634ba7f9-13bf-48e0-b52d-1767ba1d5e7a,tcga_brca
1,IDC,04689829-4071-4d50-b2dc-dbadced0e79d,tcga_brca
2,IDC,732efad0-75c8-4335-99d6-68f6e359f0ef,tcga_brca
3,IDC,f6a03071-e192-40c0-a377-27ca5307153a,tcga_brca
4,IDC,b2f025ac-8498-4e05-a9fc-b91d04737e4f,tcga_brca
...,...,...,...
28828,IDC,1864b028-bc6f-44b8-aecf-525b7bd44143,tcga_brca
28829,IDC,fdeaaabd-348a-4efb-9e0e-afab189a66f7,tcga_brca
28830,IDC,20e39936-abf6-43cf-af78-c0ebd066fb48,tcga_brca
28831,IDC,bc9995bb-f382-4d49-8b2d-45f670cde059,tcga_brca


In [167]:
pd.json_normalize(r2, record_path =['identifier'], meta= ['associated_project'], max_level = 1,errors='ignore')['system'].unique()

array(['IDC'], dtype=object)

In [168]:
pd.json_normalize(r2, record_path =['identifier'], meta= ['associated_project'], max_level = 1,errors='ignore')['associated_project'].unique()

array(['tcga_brca'], dtype=object)

In [178]:
r[0]

{'id': 'TCGA-E2-A15F',
 'identifier': [{'system': 'GDC', 'value': 'TCGA-E2-A15F'},
  {'system': 'IDC', 'value': 'TCGA-E2-A15F'}],
 'species': 'Homo sapiens',
 'sex': 'female',
 'race': 'white',
 'ethnicity': 'not hispanic or latino',
 'days_to_birth': '-23660',
 'subject_associated_project': ['TCGA-BRCA', 'tcga_brca'],
 'vital_status': 'Alive',
 'age_at_death': None,
 'cause_of_death': None,
 'File': [{'id': '62d4515f-a30b-4b1a-b2dd-c8bf9476e803',
   'identifier': [{'system': 'GDC',
     'value': '62d4515f-a30b-4b1a-b2dd-c8bf9476e803'}],
   'label': 'nationwidechildrens.org_clinical_follow_up_v4.0_brca.txt',
   'data_category': 'Clinical',
   'data_type': 'Clinical Supplement',
   'file_format': 'BCR Biotab',
   'associated_project': 'TCGA-BRCA',
   'drs_uri': 'drs://dg.4DFC:62d4515f-a30b-4b1a-b2dd-c8bf9476e803',
   'byte_size': '123240',
   'checksum': '002ad34525a69b2603c7947c53260b82',
   'data_modality': 'Genomic',
   'imaging_modality': None,
   'dbgap_accession_number': None},
  